<a href="https://colab.research.google.com/github/jana-nf/CursoSamsungOcean_Python_ConsultaSQL/blob/main/Python_Consulta_Sql_16jan_SamsungOcean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Conectando no sqlite no colab

In [ ]:
import sqlite3
con = sqlite3.connect('loja.db')
cursor = con.cursor()

cursor.execute("DROP TABLE IF EXISTS produto")
cursor.execute("DROP TABLE IF EXISTS pedido")

# 2. Criando as tabelas

In [ ]:
cursor.execute("""
CREATE TABLE pedido(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  cliente TEXT NOT NULL,
  data_pedido DATE NOT NULL,
  status TEXT NOT NULL
);
""")

con.commit()

OperationalError: table pedido already exists

In [ ]:
cursor.execute("""
CREATE TABLE produto(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  nome TEXT NOT NULL,
  preco REAL NOT NULL,
  pedido_id INTEGER NOT NULL,
  FOREIGN KEY(pedido_id) REFERENCES pedido(id)
);
""")

con.commit()

# 3. Alter Table

In [ ]:
cursor.execute("""
ALTER TABLE pedido ADD COLUMN forma_pagamento TEXT;
""")
con.commit()

# 4. Inserindo informações nas tabelas

In [ ]:
cursor.execute("""
INSERT INTO pedido(cliente, data_pedido, status, forma_pagamento)
VALUES ('Lucas Henry', '2026-01-26', 'Finalizado', 'PIX')
""")
con.commit()

In [ ]:
pedidos = [
    ("Hellen Mesquita", '2025-10-15', "Em processamento", "Cartão de Crédito"),
    ("Davi Lima", '2026-01-01', "Criado", "Boleto"),
]

cursor.executemany("""
INSERT INTO pedido (cliente, data_pedido, status, forma_pagamento)
VALUES(?, ?, ?, ?)""", pedidos)
con.commit()

In [ ]:
produtos = [
    ("Notebook", 4500.00,  1),
    ("Mouse", 150.00, 1),
    ("Teclado", 250.00, 2),
    ("Monitor", 750.00, 3)
]

cursor.executemany("""
INSERT INTO produto (nome, preco, pedido_id)
VALUES(?, ?, ?)""", produtos)
con.commit()

# 5. Buscando informações nas tabelas

In [ ]:
cursor.execute("SELECT * FROM pedido")
for linha in cursor.fetchall():
  print(linha)

(1, 'Lucas Henry', '2026-01-26', 'Finalizado', 'PIX')
(2, 'Hellen Mesquita', '2025-10-15', 'Em processamento', 'Cartão de Crédito')
(3, 'Davi Lima', '2026-01-01', 'Criado', 'Boleto')


In [ ]:
cursor.execute("SELECT * FROM pedido")
# print(cursor.fetchall())
# print(cursor.fetchmany(1))
print(cursor.fetchone())




(1, 'Lucas Henry', '2026-01-26', 'Finalizado', 'PIX')


In [ ]:
cursor.execute("""SELECT
pedido.cliente,
pedido.data_pedido,
produto.nome,
produto.preco
FROM pedido
INNER JOIN produto ON pedido.id = produto.pedido_id
""")

for linha in cursor.fetchall():
  print(linha)

('Lucas Henry', '2026-01-26', 'Notebook', 4500.0)
('Lucas Henry', '2026-01-26', 'Mouse', 150.0)
('Hellen Mesquita', '2025-10-15', 'Teclado', 250.0)
('Davi Lima', '2026-01-01', 'Monitor', 750.0)


## SQLAlchemy -> RAW

In [ ]:
from sqlalchemy import create_engine, text
engine = create_engine("sqlite:///zoo.db")
with engine.connect() as conn:
  conn.execute(text('DROP TABLE IF EXISTS animais'))

In [ ]:
with engine.begin() as conn:
  conn.execute(text("""
      CREATE TABLE animais(
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      nome TEXT,
      especie TEXT,
      idade INTEGER)
      """
      ))

In [ ]:
with engine.begin() as conn:
  conn.execute(text("""
  INSERT INTO animais(nome, especie, idade)
  VALUES ('Rex', 'Cachorro', 2)"""))

In [ ]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))
  for r in rs:
    print(r)

(1, 'Rex', 'Cachorro', 2)


In [ ]:
with engine.begin() as conn:
  conn.execute(text("""
  INSERT INTO animais (nome, especie, idade)
  VALUES
  ('Luna', 'Gato', 2),
  ('Max', 'Cachorro', 4),
  ('Bella', 'Gato', 1),
  ('Simba', 'Cachorro', 3),
  ('Oliver', 'Cachorro', 2),
  ('Milo', 'Cachorro', 1),
  ('Nala', 'Cachorro', 2)
  """))

In [ ]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))
  for r in rs:
    print(r)

(1, 'Rex', 'Cachorro', 2)
(2, 'Luna', 'Gato', 2)
(3, 'Max', 'Cachorro', 4)
(4, 'Bella', 'Gato', 1)
(5, 'Simba', 'Cachorro', 3)
(6, 'Oliver', 'Cachorro', 2)
(7, 'Milo', 'Cachorro', 1)
(8, 'Nala', 'Cachorro', 2)


In [ ]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))

  print(rs.fetchmany(3))

[(1, 'Rex', 'Cachorro', 2), (2, 'Luna', 'Gato', 2), (3, 'Max', 'Cachorro', 4)]


# SQLAlchemy Expression Language

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, select, insert
engine = create_engine("sqlite:///brasil.db")
metadata = MetaData()


In [ ]:
estados = Table(
    "estados", metadata,
    Column("id", Integer, primary_key=True),
    Column("nome", String),
    Column("sigla", String)

)

In [ ]:
metadata.create_all(engine)

In [ ]:
with engine.connect() as con:
  con.execute(insert(estados).values(nome="Pernambuco", sigla ="PE"))
  rs = con.execute(select(estados))
  print(rs.fetchone())

(1, 'Pernambuco', 'PE')


# ORM - Object Relational Mapper

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

Base = declarative_base()

In [ ]:
class Pedido(Base):
  __tablename__ = "pedidos"

  id = Column(Integer, primary_key=True)
  cliente = Column(String)

  produtos = relationship("Produto", back_populates="pedido")

In [ ]:
class Produto(Base):
    __tablename__ = "produtos"

    id = Column(Integer, primary_key=True)
    nome = Column(String)
    preco = Column(Integer)
    pedido_id = Column(Integer, ForeignKey("pedidos.id"))

    # Relacionamento com pedidos
    pedido = relationship("Pedido", back_populates="produtos")

In [ ]:
engine = create_engine("sqlite:///vendas.db")

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
pedido1 = Pedido(cliente="Danilo")
pedido2 = Pedido(cliente="Maria")

# Cria produtos
produto1 = Produto(nome="Teclado", preco=150, pedido=pedido1)
produto2 = Produto(nome="Mouse", preco=80, pedido=pedido1)
produto3 = Produto(nome="Monitor", preco=1200, pedido=pedido2)

# Adiciona tudo à sessão
session.add_all([pedido1, pedido2, produto1, produto2, produto3])
session.commit()

In [ ]:
pedidos = session.query(Pedido).all()

for pedido in pedidos:
    print(f"\nPedido {pedido.id} - Cliente: {pedido.cliente}")
    for produto in pedido.produtos:
        print(f"  Produto: {produto.nome} - R$ {produto.preco}")


Pedido 1 - Cliente: Danilo
  Produto: Teclado - R$ 150
  Produto: Mouse - R$ 80

Pedido 2 - Cliente: Maria
  Produto: Monitor - R$ 1200
